In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

from quantrocket import get_prices
from quantrocket.master import get_securities_reindexed_like


def get_qr_prices_for_ml4t(
        db_code,
        start_date, 
        end_date, 
        fields
        ):
    
    qr_prices = get_prices(db_code, 
        start_date=start_date, 
        end_date=end_date, 
        fields=fields    
    )
    
    closes = qr_prices.loc["Close"]
    securities = get_securities_reindexed_like(closes, fields=["Exchange", "Symbol"])
    prices = qr_prices.copy()
    symbols = securities.loc["Symbol"]
    prices.columns = symbols.iloc[0].tolist()
    prices.columns.name = 'ticker'
    prices.index.name = 'date'

    duplicated_columns = prices.columns[prices.columns.duplicated()]
    prices = prices.drop(columns=duplicated_columns)

    prices = prices.unstack("Field").stack('ticker').swaplevel().sort_index()
    prices.rename(columns={"Close": "close", "Open": "open", "High": "high", "Low": "low", "Volume": "volume"}, inplace=True)
    prices.index.names = [ 'ticker', 'date']
    prices.columns.name = ''
    return prices
    

In [ ]:
prices = get_qr_prices_for_ml4t(
    db_code="usstock-1d", 
    start_date="2018-01-01", 
    end_date="2018-01-05", 
    fields=["Open", "High", "Low", "Close", "Volume"]
)
prices

In [6]:
from codeload.moonshot.qr_data_ml4t import get_qr_prices_for_ml4t 
prices = get_qr_prices_for_ml4t(
    # db_code="sharadar-us-stk-1d",
    db_code="usstock-1d", 
    start_date="2018-01-01", 
    end_date="2018-01-05", 
    fields=["Open", "High", "Low", "Close", "Volume"]
)
prices

close     high      low     open     volume
ticker date                                                     
A      2018-01-02  64.8654  65.1437  64.6159  64.6927  1048026.0
       2018-01-03  66.5158  66.6789  64.8654  64.8846  1699099.0
       2018-01-04  66.0168  66.9956  65.9977  66.7269  2231719.0
       2018-01-05  67.0723  67.2643  65.9497  65.9497  1632712.0
AA     2018-01-02  54.2323  54.2815  52.9938  53.1412  2929344.0
...                    ...      ...      ...      ...        ...
ZYME   2018-01-05   8.4000   8.4999   8.4000   8.4800    11210.0
ZYNE   2018-01-02  12.9950  13.1300  12.6111  12.6200   295687.0
       2018-01-03  13.4600  14.4500  13.2000  13.2000   681984.0
       2018-01-04  12.7000  13.8899  12.0800  13.5700   658608.0
       2018-01-05  12.4950  12.8500  12.3501  12.8100   339986.0

[32267 rows x 5 columns]